In [1]:
import pandas as pd
import numpy as np
from vali_tools.valitools import check_health
pd.set_option('display.max_column',None)
import datetime
import matplotlib.pyplot as plt


In [2]:
cyc = pd.read_csv(r"Source\Data\cycles.csv")
far = pd.read_csv(r"Source\Data\farms.csv")
fas = pd.read_csv(r"Source\Data\fasting.csv")
fas_s = pd.read_csv(r"Source\Data\fastings.csv")
fet = pd.read_csv(r"Source\Data\feed_tray.csv")
fee = pd.read_csv(r"Source\Data\feeds.csv")
har = pd.read_csv(r"Source\Data\harvests.csv")
mea = pd.read_csv(r"Source\Data\measurements.csv")
mor = pd.read_csv(r"Source\Data\mortalities.csv")
pon = pd.read_csv(r"Source\Data\ponds.csv")
sam = pd.read_csv(r"Source\Data\samplings.csv")

In [3]:
cyc.sample(5)

,id,pond_id,species_id,total_seed,started_at,finished_at,remark,created_at,updated_at,area,initial_age,limit_weight_per_area,target_cultivation_day,target_size,extracted_at,subscription_type,ordered_at,hatchery_id,total_seed_type,hatchery_name,pond_name,pond_length,pond_width,pond_depth
353,7622,16052,1.0,138321,2021-01-13 00:00:00.000,2021-04-17 00:00:00.000,NaN,2021-01-12 02:50:41.000,2021-04-24 00:03:36.000,1096.0,1.0,1.25,120.0,45.0,2024-04-12 17:02:22.000,Free,NaN,NaN,NaN,NaN,D1,38.23,28.67,2.0
1996,27285,43593,NaN,27500,2023-11-26 00:00:00.000,2024-02-29 00:00:00.000,#BD-Katalis07,2023-11-06 14:39:33.000,2024-03-05 13:38:40.000,78.5,0.0,4.00,105.0,55.0,2024-04-12 17:02:22.000,Free,2023-11-06 14:39:33.000,24.0,net,PT. Prima Larvae,A4,10.00,10.00,1.0
2299,17225,24193,1.0,150000,2022-07-15 00:00:00.000,2022-10-16 00:00:00.000,NaN,2022-07-15 18:30:03.000,2022-10-23 00:09:05.000,1325.0,0.0,1.25,120.0,30.0,2024-04-12 17:02:22.000,Free,NaN,NaN,net,NaN,Bunda 2,53.00,25.00,2.0
87,12937,28053,1.0,50000,2021-10-03 00:00:00.000,2021-11-24 00:00:00.000,NaN,2021-11-04 05:29:39.000,2021-11-26 03:31:23.000,900.0,0.0,1.25,120.0,45.0,2024-04-12 17:02:22.000,Free,NaN,NaN,gross,NaN,Mutiara Timur,32.66,24.49,1.0
408,18635,12087,1.0,223000,2022-10-01 00:00:00.000,2022-12-09 00:00:00.000,#SiklusBDRijal,2022-10-03 03:03:02.000,2022-12-18 00:04:55.000,1600.0,0.0,1.25,100.0,55.0,2024-04-12 17:02:22.000,Free,NaN,1004.0,net,PT. Tri Karta Pratama - Total Kualitas Prima,A9_MUSLIH,46.19,34.64,1.6


## Note Cycle
- id should be unique
- there are duplicate on id
- features: area, initial_age,species_id,subscription_type,cycle_dur(new)

In [3]:
# cyc transform
cyc.id = cyc.id.astype(str) 
cyc.pond_id = cyc.pond_id.astype(str) 
cyc.species_id = cyc.species_id.astype(str)
#drop dupe assume cycle is unique
cyc = cyc.drop_duplicates(subset='id')

cyc.started_at = pd.to_datetime(cyc.started_at)
cyc.finished_at = pd.to_datetime(cyc.finished_at)
cyc.created_at = pd.to_datetime(cyc.created_at)
cyc.updated_at = pd.to_datetime(cyc.updated_at)
cyc.extracted_at = pd.to_datetime(cyc.extracted_at)
cyc.ordered_at = pd.to_datetime(cyc.ordered_at)
cyc.finished_at.fillna(datetime.datetime.now(),inplace=True)

cyc.species_id = cyc.species_id.replace('nan',np.nan)

cyc['cyc_dur'] = cyc.finished_at - cyc.started_at
cyc['cyc_dur'] = cyc['cyc_dur'].astype(str)
cyc['cyc_dur'] = cyc['cyc_dur'].str.split(' ').str[0]
cyc['cyc_dur'] = cyc['cyc_dur'].astype(int)
cyc.rename(columns={'id':'cycle_id'},inplace=True)

C:\Users\kevin\AppData\Local\Temp\ipykernel_28576\3497759362.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  cyc.finished_at.fillna(datetime.datetime.now(),inplace=True)


In [5]:
check_health(cyc)

,col_name,null_count,unique_count,row_count,dupe_count,null_pct,unique_pct,dupe_pct,mean,std,min,max,25_pct,50_pct,75_pct,col_types,mode,freq
0,cycle_id,0,2500,2500,0,0.0000,1.0000,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,object,20849,1
1,pond_id,0,1675,2500,825,0.0000,0.6700,0.3300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,object,24569,8
2,species_id,790,2,2500,2498,0.3160,0.0008,0.9992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,object,1.0,1576
3,total_seed,0,868,2500,1632,0.0000,0.3472,0.6528,217154.5784,170662.188169,10.0,1800000.0,90000.0,184722.0,300000.0,numeric,NaN,NaN
4,started_at,0,750,2500,1750,0.0000,0.3000,0.7000,2022-10-27 02:43:00.479999744,NaN,2020-03-07 00:00:00,2024-02-18 00:00:00,2022-05-08 00:00:00,2022-11-16 00:00:00,2023-08-05 00:00:00,numeric,NaN,NaN
5,finished_at,0,858,2500,1642,0.0000,0.3432,0.6568,2023-01-13 23:24:46.415686656,NaN,2020-05-20 00:00:00,2024-09-05 20:13:59.216932,2022-07-16 18:00:00,2023-02-10 00:00:00,2023-10-26 00:00:00,numeric,NaN,NaN
6,remark,1172,67,2500,2433,0.4688,0.0268,0.9732,NaN,NaN,NaN,NaN,NaN,NaN,NaN,object,#SiklusFarm,269
7,created_at,0,2325,2500,175,0.0000,0.9300,0.0700,2022-11-10 21:50:26.408800,NaN,2020-02-19 08:44:53,2024-03-05 13:10:13,2022-05-17 04:53:08,2022-12-02 04:03:08,2023-08-10 03:05:55.750000128,numeric,NaN,NaN
8,updated_at,0,2289,2500,211,0.0000,0.9156,0.0844,2023-03-08 00:19:24.160000,NaN,2020-06-03 02:44:52,2024-04-09 00:06:26,2022-08-25 12:30:48.500000,2023-06-05 00:03:12,2023-11-20 07:32:10,numeric,NaN,NaN
9,area,0,648,2500,1852,0.0000,0.2592,0.7408,2233.092964,12372.846032,1.02,422500.0,803.84,1368.0,2238.6825,numeric,NaN,NaN


## Note fasting
- features: fasting

In [4]:
#fasting transform
fast = pd.concat([fas,fas_s])
fast.logged_date = pd.to_datetime(fast.logged_date)
fast.cycle_id = fast.cycle_id.astype(str)
fast.fasting = fast.fasting.astype(str)
fast.cycle_id = fast.cycle_id.str.split('.').str[0]
fast.fasting = fast.fasting.str.split('.').str[0]
fast = fast.drop_duplicates(subset=['logged_date','cycle_id','fasting'])
fast.fasting.replace('nan','0',inplace=True)
fast.fasting = fast.fasting.astype(int)
fast = fast[fast.fasting ==1].sort_values(['cycle_id','logged_date'])
#feature count of fasting for fast to cycle end ratio
fast_agg = fast.groupby(['cycle_id'],as_index=False).fasting.sum()

fast_test = fast.copy()
for i in fast_test.cycle_id.unique():
    fast_test.loc[fast_test.cycle_id == i,'cons_fast_count'] =  (fast_test[fast_test.cycle_id == i]['logged_date'].diff().dt.days == 1).sum()

#feature: count of consecutive day
fast_test = fast_test.drop_duplicates('cycle_id')
fast_test = fast_test[['cycle_id','cons_fast_count']]

C:\Users\kevin\AppData\Local\Temp\ipykernel_28576\3608833421.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  fast.fasting.replace('nan','0',inplace=True)


In [11]:
fet.sample(5)

,id,logged_at,feed_logged_at,cycle_id,tray_number,feed_remaining_percent,remark,created_at,updated_at,local_feed_logged_at
152796,194131,2023-10-29 14:00:00,2023-10-29 12:00:00,23710,1,66,"0,8% pakan",2023-10-29 11:41:36,2023-10-29 15:34:50,2023-10-29 21:00:00
73766,98486,2022-11-06 05:00:00,2022-11-06 03:00:00,18615,1,0,"Pakan di anco 0,5%",2022-11-08 03:55:03,2022-11-08 03:55:03,2022-11-06 12:00:00
142754,183148,2023-09-16 01:30:00,2023-09-15 23:30:00,23734,1,0,NaN,2023-09-18 02:52:13,2023-09-18 02:52:13,2023-09-16 08:30:00
51392,70127,2022-08-19 09:00:00,2022-08-19 08:00:00,16712,2,0,NaN,2022-08-19 10:11:13,2022-08-19 10:11:13,2022-08-19 16:00:00
132739,172336,2023-08-02 06:30:00,2023-08-02 03:30:00,23734,2,0,NaN,2023-08-02 11:16:53,2023-08-02 11:16:53,2023-08-02 13:30:00


In [12]:
check_health(fet)

,col_name,null_count,unique_count,row_count,dupe_count,null_pct,unique_pct,dupe_pct,mean,std,min,max,25_pct,50_pct,75_pct,col_types,mode,freq
0,id,0,186664,186664,0,0.000000,1.000000,0.000000,123263.899429,67748.121524,498.0,285950.0,63660.75,122951.5,180322.25,numeric,NaN,NaN
1,logged_at,0,16592,186664,170072,0.000000,0.088887,0.911113,NaN,NaN,NaN,NaN,NaN,NaN,NaN,object,2022-06-12 00:30:00,125
2,feed_logged_at,0,15298,186664,171366,0.000000,0.081955,0.918045,NaN,NaN,NaN,NaN,NaN,NaN,NaN,object,2022-06-23 00:00:00,224
3,cycle_id,0,709,186664,185955,0.000000,0.003798,0.996202,20091.738193,4137.425172,12163.0,29874.0,16162.00,19253.0,23739.00,numeric,NaN,NaN
4,tray_number,0,6,186664,186658,0.000000,0.000032,0.999968,1.950221,1.077650,1.0,6.0,1.00,2.0,3.00,numeric,NaN,NaN
5,feed_remaining_percent,0,3,186664,186661,0.000000,0.000016,0.999984,10.079935,19.511848,0.0,66.0,0.00,0.0,0.00,numeric,NaN,NaN
6,remark,140283,739,186664,185925,0.751527,0.003959,0.996041,NaN,NaN,NaN,NaN,NaN,NaN,NaN,object,1% DARI PAKAN,2849
7,created_at,0,38870,186664,147794,0.000000,0.208235,0.791765,NaN,NaN,NaN,NaN,NaN,NaN,NaN,object,2022-05-17 14:52:36,120
8,updated_at,0,38949,186664,147715,0.000000,0.208658,0.791342,NaN,NaN,NaN,NaN,NaN,NaN,NaN,object,2022-05-18 13:27:06,120
9,local_feed_logged_at,0,16510,186664,170154,0.000000,0.088448,0.911552,NaN,NaN,NaN,NaN,NaN,NaN,NaN,object,2022-07-08 07:30:00,124


In [5]:
#feed transform
fee.cycle_id = fee.cycle_id.astype(str)
fee.logged_at = fee.logged_at.fillna(method='ffill')
fee.logged_at = fee.logged_at.str.split('.').str[0]
fee.logged_at = pd.to_datetime(fee.logged_at,format="%Y-%m-%d %H:%M:%S")

fee.sort_values(['cycle_id','logged_at'],inplace=True)
fee['log_date'] = fee.logged_at.dt.date
fee['log_date'] = pd.to_datetime(fee['log_date'])

#feature feed qty
feed_qty = fee.groupby(['cycle_id'],as_index=False).quantity.sum()

fee_test = fee.groupby(['cycle_id','log_date'],as_index=False)['quantity'].sum().copy()
for i in fee_test.cycle_id.unique():
    fee_test.loc[fee_test.cycle_id == i,'cons_feed_count'] =  (fee_test[fee_test.cycle_id == i]['log_date'].diff().dt.days == 1).sum()

#feature: count of consecutive day
fee_test = fee_test.drop_duplicates('cycle_id')
fee_test = fee_test[['cycle_id','cons_feed_count']]

feed = feed_qty.merge(fee_test,how='left',on='cycle_id')

C:\Users\kevin\AppData\Local\Temp\ipykernel_28576\3522420325.py:3: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  fee.logged_at = fee.logged_at.fillna(method='ffill')


In [17]:
mea.sample(5)

,pond_id,cycle_id,measured_date,morning_temperature,evening_temperature,morning_do,evening_do,morning_salinity,evening_salinity,morning_pH,evening_pH,transparency,turbidity,ammonia,nitrate,nitrite,alkalinity,hardness,calcium,magnesium,carbonate,bicarbonate,tom,total_plankton_
47532,39087,21203,2023-04-25,28.65,NaN,2.48,NaN,19.35,NaN,8.14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51595,44540,29100,2023-08-21,32.00,NaN,4.60,NaN,36.00,NaN,9.00,NaN,60.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71040,46258,28067,2024-01-05,28.60,29.5,5.30,5.5,28.00,28.0,7.72,7.75,45.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
136926,40661,25307,2023-09-15,NaN,NaN,NaN,NaN,NaN,NaN,8.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
134682,45257,27088,2023-10-18,NaN,NaN,NaN,NaN,36.00,NaN,7.90,8.20,25.0,NaN,NaN,10.0,0.0,176.0,NaN,NaN,NaN,0.0,176.0,83.0,3290000.0


In [6]:
mea.cycle_id = mea.cycle_id.astype(str)
mea.pond_id = mea.pond_id.astype(str)
mea['measured_date'] = pd.to_datetime(mea['measured_date'])
#bak df
mea_ori = mea.copy()
mea[['morning_temperature', 'evening_temperature', 'morning_do',
       'evening_do', 'morning_salinity', 'evening_salinity', 'morning_pH',
       'evening_pH', 'transparency', 'turbidity', 'ammonia', 'nitrate',
       'nitrite', 'alkalinity', 'hardness', 'calcium', 'magnesium',
       'carbonate', 'bicarbonate', 'tom', 'total_plankton_']] = mea[['morning_temperature', 'evening_temperature', 'morning_do',
       'evening_do', 'morning_salinity', 'evening_salinity', 'morning_pH',
       'evening_pH', 'transparency', 'turbidity', 'ammonia', 'nitrate',
       'nitrite', 'alkalinity', 'hardness', 'calcium', 'magnesium',
       'carbonate', 'bicarbonate', 'tom', 'total_plankton_']].fillna(0)

mea_agg = mea.groupby('cycle_id',as_index=False).agg({'morning_temperature':'mean',
                                                      'evening_temperature':'mean',
                                                      'morning_do':'mean',
                                                      'morning_salinity':'mean',
                                                      'evening_salinity':'mean',
                                                      'morning_pH':'mean',
                                                      'evening_pH':'mean',
                                                      'transparency':'mean',
                                                      'turbidity':'mean',
                                                      'ammonia':'mean',
                                                      'nitrate':'mean',
                                                      'nitrite':'mean',
                                                      'alkalinity':'mean',
                                                      'hardness':'mean',
                                                      'calcium':'mean',
                                                      'magnesium':'mean',
                                                      'carbonate':'mean',
                                                      'bicarbonate':'mean',
                                                      'tom':'mean',
                                                      'total_plankton_':'mean'})

In [24]:
mea_agg[mea_agg.cycle_id == '19874']

,cycle_id,morning_temperature,evening_temperature,morning_do,morning_salinity,evening_salinity,morning_pH,evening_pH,transparency,turbidity,ammonia,nitrate,nitrite,alkalinity,hardness,calcium,magnesium,carbonate,bicarbonate,tom,total_plankton_
1082,19874,27.669371,30.088828,3.411747,14.568344,14.255043,7.980608,8.127548,64.650538,0.0,0.0,0.0,0.007366,18.827957,400.16129,31.913978,86.129032,2.44086,17.311828,8.226559,8713.010753


In [19]:
mor.sample(5)

,id,cycle_id,quantity,recorded_at,created_at,updated_at,average_weight
1213,6619,15495,4695,2022-06-30,2022-07-02 07:50:45,2024-01-08 11:40:40,4.26
6048,13339,18675,72,2023-01-18,2023-01-25 15:08:17,2024-01-08 11:40:40,24.39
11094,20856,26245,60,2023-11-28,2023-12-06 08:07:46,2023-12-06 08:07:47,10.58
12624,23093,27729,140,2024-02-08,2024-02-14 18:26:02,2024-02-14 18:26:08,14.29
1916,7610,16469,1036,2022-07-23,2022-07-24 02:18:19,2024-01-08 11:40:40,1.93


In [20]:
check_health(mor)

,col_name,null_count,unique_count,row_count,dupe_count,null_pct,unique_pct,dupe_pct,mean,std,min,max,25_pct,50_pct,75_pct,col_types,mode,freq
0,id,0,13221,13221,0,0.0,1.000000,0.000000,14307.947810,5630.215217,185.0,26192.00,9763.00,13998.00,18858.00,numeric,NaN,NaN
1,cycle_id,0,857,13221,12364,0.0,0.064821,0.935179,20844.158763,4098.897296,7273.0,29579.00,17726.00,19676.00,24092.00,numeric,NaN,NaN
2,quantity,0,2538,13221,10683,0.0,0.191967,0.808033,1834.642085,87210.179102,1.0,10000000.00,60.00,200.00,756.00,numeric,NaN,NaN
3,recorded_at,0,896,13221,12325,0.0,0.067771,0.932229,NaN,NaN,NaN,NaN,NaN,NaN,NaN,object,2023-10-13,55
4,created_at,0,11340,13221,1881,0.0,0.857726,0.142274,NaN,NaN,NaN,NaN,NaN,NaN,NaN,object,2023-01-24 04:36:05,27
5,updated_at,0,1475,13221,11746,0.0,0.111565,0.888435,NaN,NaN,NaN,NaN,NaN,NaN,NaN,object,2024-01-08 11:40:40,11713
6,average_weight,0,1984,13221,11237,0.0,0.150064,0.849936,9.613240,17.196472,0.0,1428.57,4.55,7.89,13.04,numeric,NaN,NaN


In [7]:
#pond + farm Transform
pon.farm_id = pon.farm_id.astype(str)
pon.id = pon.id.astype(str)
pon.rename(columns={'id':'pond_id'},inplace=True)

pon.created_at = pd.to_datetime(pon.created_at)
pon.updated_at = pd.to_datetime(pon.updated_at)
pon.extracted_at = pd.to_datetime(pon.extracted_at)

pon.length = pon.length.astype(float)
pon.width = pon.width.astype(float)
pon.deep = pon.deep.astype(float)
pon.max_seed_density = pon.max_seed_density.astype(float)

far.id = far.id.astype(str)
far['h_tz'] = far.timezone.str.replace('+','').str.split(':').str[0]
far['h_tz'] = far['h_tz'].astype(int)
far['mnt_tz'] = far.timezone.str.replace('+','').str.split(':').str[1]
far['mnt_tz'] = far['mnt_tz'].astype(int)
far.rename(columns={'id':'farm_id'},inplace=True)
#merge pond with farm
pon_far = pon.merge(far,how='left',on='farm_id')
pon_far = pon_far[['pond_id','length','width','deep','max_seed_density','province','regency','timezone','h_tz','mnt_tz']]

In [8]:
#sampling + harvest transform
#manual correction w/ assum on created_at date
sam.loc[sam.sampled_at == '1-01-01','sampled_at'] = '2023-10-12'

sam.updated_at = pd.to_datetime(sam.updated_at,format="%Y-%m-%d %H:%M:%S")
sam.sampled_at = pd.to_datetime(sam.sampled_at,format="%Y-%m-%d")
sam.created_at = pd.to_datetime(sam.created_at,format="%Y-%m-%d %H:%M:%S")

sam.average_weight = sam.average_weight.astype(float)
sam.id = sam.id.astype(str)
sam.cycle_id = sam.cycle_id.astype(str)

sam.cycle_id = sam.cycle_id.str.split('.').str[0]
sam.id = sam.id.str.split('.').str[0]

#get last updated sampling record
sam.sort_values(by=['id','updated_at'],inplace=True)
sam.drop_duplicates(subset='id',keep='last',inplace=True)
#get cycle last sample
sam.sort_values(by=['cycle_id','sampled_at'],inplace=True)
sam.drop_duplicates(subset='cycle_id',keep='last',inplace=True)

har.id = har.id.astype(str)
har.cycle_id = har.cycle_id.astype(str)

har.cycle_id = har.cycle_id.str.split('.').str[0]
har.id = har.id.str.split('.').str[0]

har.updated_at = pd.to_datetime(har.updated_at,format="%Y-%m-%d %H:%M:%S")
har.created_at = pd.to_datetime(har.created_at,format="%Y-%m-%d %H:%M:%S")
har.harvested_at = pd.to_datetime(har.harvested_at,format="%Y-%m-%d")
har.selling_price.fillna(0,inplace=True)

#get non failed harvest
har = har[har.status !='Failed']

#standardize weigth metric
har['avg_weight_har'] = 1000/har['size']

#aggregate partial harvest
har_agg = har.groupby('cycle_id',as_index=False)\
    .agg({'weight':'sum','size':'mean','updated_at':'max','selling_price':'sum'})
#standardize weigth metric
har_agg['avg_weight_har'] = 1000/har_agg['size']
sam_har = har_agg.merge(sam,how='left',on='cycle_id',suffixes=('_har','_sam'))\
    .drop(columns=['created_at','updated_at_sam','updated_at_har','id','remark'])

#analyze correlation of last sampling avg weight with harvest avg weight
#strongly correlated, means the target can only be used interchangeably
# sam_har[sam_har.average_weight.notnull()].average_weight.corr(sam_har[sam_har.average_weight.notnull()].avg_weight_har,method='pearson')
# sam_har.plot.scatter(x='average_weight',y='avg_weight_har')

C:\Users\kevin\AppData\Local\Temp\ipykernel_28576\1262045422.py:32: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  har.selling_price.fillna(0,inplace=True)


In [26]:
#merge to cycle
features = cyc.merge(fast_agg,how='left',on='cycle_id')\
    .merge(fast_test,how='left',on='cycle_id')\
        .merge(feed,how='left',on='cycle_id')\
            .merge(pon_far,how='left',on='pond_id')\
                .merge(mea_agg,how='left',on='cycle_id')\
                    .merge(sam_har,how='left',on='cycle_id')


In [27]:
features

,cycle_id,pond_id,species_id,total_seed,started_at,finished_at,remark,created_at,updated_at,area,initial_age,limit_weight_per_area,target_cultivation_day,target_size,extracted_at,subscription_type,ordered_at,hatchery_id,total_seed_type,hatchery_name,pond_name,pond_length,pond_width,pond_depth,cyc_dur,fasting,cons_fast_count,quantity,cons_feed_count,length,width,deep,max_seed_density,province,regency,timezone,h_tz,mnt_tz,morning_temperature,evening_temperature,morning_do,morning_salinity,evening_salinity,morning_pH,evening_pH,transparency,turbidity,ammonia,nitrate,nitrite,alkalinity,hardness,calcium,magnesium,carbonate,bicarbonate,tom,total_plankton_,weight,size,selling_price,avg_weight_har,sampled_at,average_weight
0,18876,36294,1.0,331920,2022-10-14,2023-01-29,NaN,2022-10-14 12:34:23,2023-01-30 01:46:27,4030.0,0.0,2.00,110.0,35.0,2024-04-12 17:02:22,Free,NaT,66.0,net,UD. Benur Ndaru Laut,H,65.00,62.00,1.0,107,22.0,7.0,4660.0,106.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.519626,22.496573,3.722430,9.280374,0.869159,7.222430,1.755140,45.327103,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1853.50,91.500000,9.968670e+07,10.928962,2023-01-28,12.03
1,22118,37102,1.0,40000,2023-04-26,2023-06-23,#TBR01,2023-04-30 22:28:13,2023-06-26 05:35:32,399.0,0.0,1.25,120.0,100.0,2024-04-12 17:02:22,Free,NaT,83.0,net,CV Raja Benur,A1,30.00,20.00,1.0,58,1.0,0.0,361.5,57.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.280476,20.604524,3.481667,20.328571,14.686905,7.774524,0.820000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,270.00,110.000000,1.296000e+07,9.090909,2023-06-23,9.00
2,24088,39154,NaN,357570,2023-08-01,2023-10-18,#SiklusFarm,2023-07-12 01:13:05,2023-10-19 04:23:11,2000.0,0.0,1.25,120.0,20.0,2024-04-12 17:02:22,Free,2023-07-12 01:13:05,1004.0,gross,PT. Tri Karta Pratama - Total Kualitas Prima,A3,50.00,40.00,1.4,78,2.0,0.0,4806.5,76.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.202981,30.359206,5.608708,29.510831,28.864215,7.576966,0.618161,68.916667,0.0,0.000000,0.011111,0.008200,17.044444,680.555556,119.855556,575.700000,0.000000,15.777778,8.455556,212305.555556,2929.70,90.275000,1.644398e+08,11.077264,2023-10-17,17.07
3,17743,20914,1.0,168275,2022-07-19,2022-09-29,#SiklusCustomerPermadi,2022-08-14 04:40:46,2022-10-06 00:09:36,1512.0,0.0,1.25,100.0,55.0,2024-04-12 17:02:22,Free,NaT,3.0,actual,PT. Central Proteinaprima Tbk,B4,54.00,28.00,1.5,72,NaN,NaN,2328.0,70.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.531507,0.000000,4.436712,23.849315,0.000000,7.788767,0.000000,45.205479,0.0,0.005370,0.000000,0.030822,17.671233,126.602740,50.753425,140.136986,0.821918,16.849315,13.945205,66952.191781,1496.54,85.900000,0.000000e+00,11.641444,2022-09-26,13.45
4,17125,34187,1.0,188000,2022-07-07,2022-09-20,#SiklusTebar,2022-07-12 00:24:30,2022-10-15 04:27:24,1225.0,0.0,1.25,100.0,55.0,2024-04-12 17:02:22,Free,2022-07-12 00:24:30,35.0,net,PT Windu Alam Sentosa,A,35.00,35.00,1.2,75,3.0,1.0,1971.1,73.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.976260,28.492253,5.016713,13.450133,13.420467,7.712487,1.862840,0.000000,0.0,0.006827,0.000000,0.044667,9.813333,176.000000,29.333333,146.666667,0.160000,9.733333,4.440000,22466.666667,262.00,106.500000,1.492800e+07,9.389671,2022-09-06,6.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495,19131,34712,2.0,300000,2022-10-25,2023-01-21,NaN,2022-10-25 10:13:31,2023-01-25 08:01:19,4999.9,0.0,1.25,88.0,55.0,2024-04-12 17:02:22,Free,NaT,1020.0,actual,MATERNIDADE NACALA,E01,70.71,70.71,1.5,88,NaN,NaN,7003.0,88.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.966236,29.223502,4.410918,4.067416,0.067416,7.495993,7.690262,41.853933,0.0,0.000000,0.000000,0.097528,23.820225,0.000000,0.000000,0.000000,0.000000,0.000000,0